In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "7.Papilionidae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Baronia_brevicornis
Exon_1 Match
[['Papilio_machaon', ['29111', '29207']], ['Papilio_xuthus', ['29111', '29207']], ['Danaus_plexippus', ['29111', '29207']]]
Exon_2 Match
[['Papilio_machaon', ['29298', '29848']], ['Papilio_xuthus', ['29298', '29848']], ['Danaus_plexippus', ['29298', '29848']]]
Exon_3 Match
[['Papilio_machaon', ['30028', '30181']], ['Papilio_xuthus', ['30028', '30181']], ['Danaus_plexippus', ['30028', '30181']]]
Exon_4 mismatch
[['Papilio_machaon', ['31683', '31924']], ['Papilio_xuthus', ['31683', '31933']], ['Danaus_plexippus', ['31683', '31924']]]
Proceed?y
Papilio_machaon Exon_4 36_frame2 1.141 
SKDSSPILTSDPTSDSSDPEGSGKSNDDPSSVNNDWDASSRNNMNENTSQSCQSQQSGEKAFESSENDSLAGNNNDINIK
Papilio_xuthus Exon_4 36_frame2 0.985 
SKDSSPILTSDPTSDSSDPEGSGKSNDDPSSVNNDWDASSRNNMNENTSQSCQSQQSGEKAFESSENDSLAGNNNDINIKVDK
Danaus_plexippus Exon_4 36_frame2 1.062 
SKDSSPILTSDPTSDSSDPEGSGKSNDDPSSVNNDWDASSRNNMNENTSQSCQSQQSGEKAFESSENDSLAGNNNDINIK
Check Alignment??y
CLUSTAL format alignment by MAFFT 

Exon_1 Match
[['Papilio_machaon', ['8503553', '8503649']], ['Papilio_xuthus', ['8503553', '8503649']], ['Danaus_plexippus', ['8503553', '8503649']]]
Exon_2 Match
[['Papilio_machaon', ['8503761', '8504311']], ['Papilio_xuthus', ['8503761', '8504311']], ['Danaus_plexippus', ['8503761', '8504311']]]
Exon_3 Match
[['Papilio_machaon', ['8504408', '8504561']], ['Papilio_xuthus', ['8504408', '8504561']], ['Danaus_plexippus', ['8504408', '8504561']]]
Exon_4 Match
[['Papilio_machaon', ['8506284', '8506537']], ['Papilio_xuthus', ['8506284', '8506537']], ['Danaus_plexippus', ['8506284', '8506537']]]
Exon_5 Match
[['Papilio_machaon', ['8506627', '8506901']], ['Papilio_xuthus', ['8506627', '8506901']], ['Danaus_plexippus', ['8506627', '8506901']]]
Exon_6 Match
[['Papilio_machaon', ['8506998', '8507352']], ['Papilio_xuthus', ['8506998', '8507352']], ['Danaus_plexippus', ['8506998', '8507352']]]
Exon_7 Match
[['Papilio_machaon', ['8507427', '8507618']], ['Papilio_xuthus', ['8507427', '8507618']], ['D

Choose Species 1 - 3n
Exon_5 Match
[['Papilio_machaon', ['8146063', '8146352']], ['Papilio_xuthus', ['8146063', '8146352']], ['Danaus_plexippus', ['8146063', '8146352']]]
Exon_6 Match
[['Papilio_machaon', ['8146440', '8146794']], ['Papilio_xuthus', ['8146440', '8146794']], ['Danaus_plexippus', ['8146440', '8146794']]]
Exon_7 Match
[['Papilio_machaon', ['8146869', '8147060']], ['Papilio_xuthus', ['8146869', '8147060']], ['Danaus_plexippus', ['8146869', '8147060']]]
Exon_8 Match
[['Papilio_machaon', ['8147360', '8147460']], ['Papilio_xuthus', ['8147360', '8147460']], ['Danaus_plexippus', ['8147360', '8147460']]]
Exon_9 Match
[['Papilio_machaon', ['8147547', '8147745']], ['Papilio_xuthus', ['8147547', '8147745']], ['Danaus_plexippus', ['8147547', '8147745']]]
Exon_10 Match
[['Papilio_machaon', ['8148215', '8148426']], ['Papilio_xuthus', ['8148215', '8148426']], ['Danaus_plexippus', ['8148215', '8148426']]]
Exon_11 Match
[['Papilio_machaon', ['8149233', '8149705']], ['Papilio_xuthus', ['81

Choose Species 1 - 3n
Exon_16 Match
[['Papilio_machaon', ['2914353', '2914530']], ['Papilio_xuthus', ['2914353', '2914530']], ['Danaus_plexippus', ['2914353', '2914530']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Luehdorfia_chinensis,JAHVAK010000103.1,2925207,2925303,1,N,Papilio_xuthus_XM_013325762.1_Frame_0_rightoh_1_query_Exon_1,1,32,32
Luehdorfia_chinensis,JAHVAK010000103.1,2924553,2925103,1,N,Papilio_machaon_XM_045679415.1_Frame_2_rightoh_0_query_Exon_2,1,183,183,Y,N
Luehdorfia_chinensis,JAHVAK010000103.1,2924268,2924421,1,N,Papilio_machaon_XM_045679415.1_Frame_0_rightoh_1_query_Exon_3,1,51,51,Y,N
Luehdorfia_chinensis,JAHVAK010000103.1,2923626,2923867,1,N,Papilio_machaon_XM_045679415.1_Frame_2_rightoh_0_query_Exon_4,1,80,80,Y,N
Luehdorfia_chinensis,JAHVAK010000103.1,2923275,2923549,1,N,Papilio_machaon_XM_045679415.1_Frame_0_rightoh_2_query_Exon_5,1,91,91,Y,N
Luehdorfia_chinensis,JAHVAK010000103.1,2922849,2923203,1,N,Papilio_machaon_XM_04

Exon_1 Match
[['Papilio_machaon', ['1250796', '1250892']], ['Papilio_xuthus', ['1250796', '1250892']], ['Danaus_plexippus', ['1250796', '1250892']]]
Exon_2 Match
[['Papilio_machaon', ['1249433', '1249983']], ['Papilio_xuthus', ['1249433', '1249983']], ['Danaus_plexippus', ['1249433', '1249983']]]
Exon_3 Match
[['Papilio_machaon', ['1249136', '1249289']], ['Papilio_xuthus', ['1249136', '1249289']], ['Danaus_plexippus', ['1249136', '1249289']]]
Exon_4 mismatch
[['Papilio_machaon', ['1248428', '1248726']], ['Papilio_xuthus', ['1248473', '1248726']], ['Danaus_plexippus', ['1248440', '1248726']]]
Proceed?y
Papilio_machaon Exon_4 16_frame2 1.213 
SKDSSPNLTSDPTSDSSDTSGSGKSNVDPSSMNNAWDMSARNNGNKNSSPSFQSPREVDTKITKLYENTNCEVTNKDATVKTGEVSYFSFKPIHIVRIL
Papilio_xuthus Exon_4 16_frame2 1.157 
SKDSSPNLTSDPTSDSSDTSGSGKSNVDPSSMNNAWDMSARNNGNKNSSPSFQSPREVDTKITKLYENTNCEVTNKDATVKTGE
Danaus_plexippus Exon_4 16_frame2 1.156 
SKDSSPNLTSDPTSDSSDTSGSGKSNVDPSSMNNAWDMSARNNGNKNSSPSFQSPREVDTKITKLYENTNCEVTNKDATVKTGEVS

...

Exon_1 Match
[['Papilio_machaon', ['4221886', '4221982']], ['Papilio_xuthus', ['4221886', '4221982']], ['Danaus_plexippus', ['4221886', '4221982']]]
Exon_2 Match
[['Papilio_machaon', ['4222087', '4222637']], ['Papilio_xuthus', ['4222087', '4222637']], ['Danaus_plexippus', ['4222087', '4222637']]]
Exon_3 Match
[['Papilio_machaon', ['4222780', '4222933']], ['Papilio_xuthus', ['4222780', '4222933']], ['Danaus_plexippus', ['4222780', '4222933']]]
Exon_4 Match
[['Papilio_machaon', ['4223201', '4223460']], ['Papilio_xuthus', ['4223201', '4223460']], ['Danaus_plexippus', ['4223201', '4223460']]]
Exon_5 Match
[['Papilio_machaon', ['4223539', '4223816']], ['Papilio_xuthus', ['4223539', '4223816']], ['Danaus_plexippus', ['4223539', '4223816']]]
Exon_6 Match
[['Papilio_machaon', ['4223893', '4224247']], ['Papilio_xuthus', ['4223893', '4224247']], ['Danaus_plexippus', ['4223893', '4224247']]]
Exon_7 Match
[['Papilio_machaon', ['4224350', '4224541']], ['Papilio_xuthus', ['4224350', '4224541']], ['D

Proceed?y
Papilio_machaon Exon_15 15_frame0 1.313 
TIIKEEKIEVKVTEVKTQLGPERVLEGLWLARGLAAGAVPGVVAGAMPGEEEMPMPDAALPAAMAAMAAMAAKPEINYNHM
Papilio_xuthus Exon_15 15_frame0 1.098 
TIIKEEKIEVKVTEVKTQLGPERVLEGLWLARGLAAGAVPGVVAGAMPGEEEMPMPDAALPAAMAAMAAMAAKPEINYNHM
Danaus_plexippus Exon_15 15_frame0 1.659 
TIIKEEKIEVKVTEVKTQLGPERVLEGLWLARGLAAGAVPGVVAGAMPGEEEMPMPDAALPAAMAAMAAMAAKPEINYNHMESEYFNT
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon -------SIVKEEMVEVKVTEVKGR-SDQEVREERGRVRGQEAGAEAGAEA-----EEAS
set15_frame0    YLNGIFSTIIKEEKIEVKVTEVKTQLGPERVLEGLWLARGLAAGAVPGVVAGAMPGEEEM
                       :*:*** :******** : . :.* *    .**  *** .*. *     **  

Papilio_machaon CMPDAALAIPAATSATSATTTTSTKPDVKYNLHLMESEYHN
set15_frame0    PMPDAAL--PAAMAAMAA---MAAKPEINYN--HMESEYFN
                 ******  *** :* :*    ::**:::**   *****.*

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_xuthus_ -------TILKEEIIEVKVTEVQGRVGEEGVREEAGRA-GRARGQEAG--AGEGPEEEET
set15

Proceed?y
Papilio_machaon Exon_15 10_frame2 0.744 
TILKEEIIEVKVTEVQGRVGEEGVREEAGRAGRARGQEAGAGEGPEEEE
Papilio_xuthus Exon_15 10_frame2 0.0 
TILKEEIIEVKVTEVQGRVGEEGVREEAGRAGRARGQEAGAGEGPEEEETSCMPDAALAMPAAMAAMSAKPDVNYNLHHMESEYYN
Danaus_plexippus Exon_15 10_frame2 1.766 
TILKEEIIEVKVTEVQGRVGEEGVREEAGRAGRARGQEAGAGEGPEEEETSCMPDAALAMPAAMAAMSAKPDVNYNLHHM
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon -----SIVKEEMVEVKVTEVKGRSDQE-VREE---RGRVRGQEAGAEAGAEAEEASCMPD
set10_frame2    NNIFSTILKEEIIEVKVTEVQGRVGEEGVREEAGRAGRARGQEAGAGEGPEEEETSCMPD
                     :*:***::*******:** .:* ****    **.*******  *.* **:*****

Papilio_machaon AALAIPAATSATSATTTTSTKPDVKYNLHLMESEYHN
set10_frame2    AALAMPAAMAAMSA------KPDVNYNLHHMESEYYN
                ****:*** :* **      ****:**** *****:*

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_xuthus_ -----TILKEEIIEVKVTEVQGRVGEEGVREEAGRAGRARGQEAGAGEGPEEEETSCMPD
set10_frame2    NNIFSTILKEEIIEVKVTEVQGRVGEEGVREEAGRAGR

Exon_1 Match
[['Papilio_machaon', ['6393590', '6393686']], ['Papilio_xuthus', ['6393590', '6393686']], ['Danaus_plexippus', ['6393590', '6393686']]]
Exon_2 Match
[['Papilio_machaon', ['6394504', '6395054']], ['Papilio_xuthus', ['6394504', '6395054']], ['Danaus_plexippus', ['6394504', '6395054']]]
Exon_3 Match
[['Papilio_machaon', ['6395211', '6395364']], ['Papilio_xuthus', ['6395211', '6395364']], ['Danaus_plexippus', ['6395211', '6395364']]]
Exon_4 mismatch
[['Papilio_machaon', ['6396595', '6396851']], ['Papilio_xuthus', ['6396595', '6396836']], ['Danaus_plexippus', ['6396595', '6396851']]]
Proceed?y
Papilio_machaon Exon_4 39_frame2 1.108 
SKDSSPNLTSNPTSDSSDTGGSGKSNEDLSSMNNDWDASLRNNMKENSGQLFQLPRLNEKIGSQKSENNNCGDRKMKTDKVRYFL
Papilio_xuthus Exon_4 39_frame2 0.984 
SKDSSPNLTSNPTSDSSDTGGSGKSNEDLSSMNNDWDASLRNNMKENSGQLFQLPRLNEKIGSQKSENNNCGDRKMKTDK
Danaus_plexippus Exon_4 39_frame2 1.209 
SKDSSPNLTSNPTSDSSDTGGSGKSNEDLSSMNNDWDASLRNNMKENSGQLFQLPRLNEKIGSQKSENNNCGDRKMKTDKVRYFL
Check Alignment??y

Exon_1 Match
[['Papilio_machaon', ['7704428', '7704524']], ['Papilio_xuthus', ['7704428', '7704524']], ['Danaus_plexippus', ['7704428', '7704524']]]
Exon_2 Match
[['Papilio_machaon', ['7703043', '7703590']], ['Papilio_xuthus', ['7703043', '7703590']], ['Danaus_plexippus', ['7703043', '7703590']]]
Exon_3 Match
[['Papilio_machaon', ['7702764', '7702917']], ['Papilio_xuthus', ['7702764', '7702917']], ['Danaus_plexippus', ['7702764', '7702917']]]
Exon_4 Match
[['Papilio_machaon', ['7702074', '7702327']], ['Papilio_xuthus', ['7702074', '7702327']], ['Danaus_plexippus', ['7702074', '7702327']]]
Exon_5 Match
[['Papilio_machaon', ['7701704', '7701972']], ['Papilio_xuthus', ['7701704', '7701972']], ['Danaus_plexippus', ['7701704', '7701972']]]
Exon_6 Match
[['Papilio_machaon', ['7701274', '7701628']], ['Papilio_xuthus', ['7701274', '7701628']], ['Danaus_plexippus', ['7701274', '7701628']]]
Exon_7 Match
[['Papilio_machaon', ['7700674', '7700865']], ['Papilio_xuthus', ['7700674', '7700865']], ['D

In [ ]:
1